In [1]:
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Binance API REST Call without api keys

def millisecond(date, place=10 ** 6):
    return pd.Timestamp(date).value // place

def url(symbol, interval, start=None, end=None, limit=1000):
    # Look back data of 3 years
    # interval: m, d, M
    # Max limit: 1000
    base_url = "https://api.binance.us/api/v3/klines?"
    symbol = "symbol=" + symbol
    interval = "&interval=" + interval
    start = "&startTime=" + str(millisecond(start)) if start is not None else ""
    end = "&endTime=" + str(millisecond(end)) if end is not None else ""
    limit = "&limit=" + str(limit)
    return base_url + symbol + interval + start + end + limit

def get_data(symbol, interval, start=None, end=None, limit=1000):
    raw = pd.read_json(url(symbol, interval, start=start, end=end, limit=limit))
    # Making change to DataFrame
    date = pd.to_datetime(raw.iloc[:, 0], unit='ms')
    date.name = None
    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = raw.iloc[:, 1:6]
    df.columns = columns
    df.index = date
    return df

In [3]:
# Load qyld csv
pd.read_csv('qyld_full-holdings_20220228.csv')

,% of Net Assets,Ticker,Name,SEDOL,Market Price ($),Shares Held,Market Value ($)
0,12.79,AAPL,APPLE INC,2046251,165.12,"5,059,138.00","835,364,866.56"
1,10.59,MSFT,MICROSOFT CORP,2588173,298.79,"2,315,126.00","691,736,497.54"
2,7.37,AMZN,AMAZON.COM INC,2000019,"3,071.26","156,642.00","481,088,308.92"
3,4.14,TSLA,TESLA INC,B616C79,870.43,"310,536.00","270,299,850.48"
4,4.05,NVDA,NVIDIA CORP,2379504,243.85,"1,085,828.00","264,779,157.80"
...,...,...,...,...,...,...,...
98,0.18,SWKS,SKYWORKS SOLUTIONS INC,2961053,138.17,"85,943.00","11,874,744.31"
99,0.18,DOCU,DOCUSIGN INC,BFYT7B7,118.43,"98,271.00","11,638,234.53"
100,0.17,PDD,PINDUODUO INC-ADR,BYVW0F7,51.86,"207,277.00","10,749,385.22"
101,0.14,SPLK,SPLUNK INC,B424494,118.10,"78,121.00","9,226,090.10"


In [4]:
qlyd = pd.read_csv('qyld_full-holdings_20220228.csv').dropna()

In [5]:
# Download the stock data
import os

folder = 'QLYD'
path = os.path.join(os.getcwd(), folder)

for ticker in qlyd.loc[:, 'Ticker']:
    try: 
        if not os.path.exists(os.path.join(path, ticker + '.csv')):
            DataLoader(ticker, dname='QLYD')
    except Exception as e:
        print(e)

# Analysis

### Periodic Gain Analysis

In [6]:
def batching(data, period):
    batches = len(data) // period
    max_length = batches * period
    return np.array(data)[-max_length:].reshape(-1, period)

def regroup(data, period):
    opens = batching(data['Open'], period)[:, 0]
    high = batching(data['High'], period).max(1)
    low = batching(data['Low'], period).min(1)
    close = batching(data['Close'], period)[:, -1]
    volume = batching(data['Volume'], period).sum(1)
    date = pd.to_datetime(batching(data.index, period)[:, -1])
    return pd.DataFrame(np.stack([opens, high, low, close, volume], axis=1),
                        columns=['Open', 'High', 'Low', 'Close', 'Volume'],
                        index=date)

In [7]:
def get_range(m, s=0.00, e=0.201, step=0.005):
    rang = np.arange(s, e, step=step)
    return np.unique(np.stack([m - rang, m + rang]))

def create_table(data, rang):
    table = {r: {'Above': (data > r).mean(),
                 'Below': (data <= r).mean()}
             for r in rang}
    return pd.DataFrame.from_dict(table, orient='index')

In [8]:
# Calculate Periodic Gains
period = 5
sect = 4
dic = {}
for ticker in qlyd.loc[:, 'Ticker']:
    # Load price data form database
    data = DataLoader(ticker, dname='QLYD').get_data('2016-01-01', '2020-12-31')
    if len(data) < (250 * 5):
        continue
    # Regrouping price data base on the period
    prices = regroup(data, period)
    # Calculate percentage 
    gain = prices['High'].shift(-1) / prices['Close'] - 1
    base = prices['Close'].shift(-1) / prices['Close'] - 1
    mgain = np.round(gain.mean(), 3)
    # best mean
    arange = np.arange(0.00, 1., step=0.001)
    best_mean = 0
    for r in (mgain - arange):
        if (gain > r).mean() > .75 and r > 0:
            best_mean = r
            break
    # replacing
    pct = gain.copy()
    mask = base <= best_mean
    pct[mask] = base[mask]
    pct.dropna(inplace=True)
    # Capital Calculate
    total_capital_gain = np.prod(pct + 1) - 1
    # Section Capital 
    sect_capital_gain = np.prod(batching(pct + 1, sect), axis=1).mean() - 1
    # Append to dictionary
    dic[ticker] = {'Mean Gain': mgain, 
                   'Best Gain': best_mean,
                   'Total Cap Gain': total_capital_gain,
                   'Sect Cap Gain': sect_capital_gain}


In [9]:
analysis = pd.DataFrame.from_dict(dic, orient='index')

In [10]:
# Create gain and base table for probability
dic = {}
for pick in analysis.index:
    data = DataLoader(pick).get_data('2016-01-01', '2020-12-31')
    prices = regroup(data, period)
    # Gains
    gain = (prices['High'].shift(-1) / prices['Close'] - 1).dropna()
    base = (prices['Close'].shift(-1) / prices['Close'] - 1).dropna()
    bgain = np.round(analysis.loc[pick, 'Best Gain'], 3)
    rang = get_range(bgain)
    dic[pick] = {'Medium': bgain,
                 'Gain Table': create_table(gain, rang),
                 'Base Table': create_table(base, rang)}

In [11]:
score = {}
for ticker in analysis.index:
    gain_table = dic[ticker]['Gain Table']
    data = DataLoader(ticker).get_data('2021-01-01', '2021-12-31')

    holding = False
    selling = False
    capital = 1000
    base_capital = capital
    share = 0
    max_share = 100
    buy_at = 0
    num = 0
    num_trades = 0

    for date, prices in data.iterrows():
        if not holding:
            price = prices['Close']
            buy_at = price
            share = capital // buy_at
            share = max_share if share > max_share else share
            holding = True
            capital -= buy_at * share
        else: 
            high = prices['High']
            close = prices['Close']
            
            pct = high / buy_at - 1
            check = gain_table.index < pct
            if check.sum() == 0:
                check[0] = True
            check_table = gain_table.loc[check, 'Above'].iloc[-1]
            if check_table < .8:
                selling = True
                sell_price = high
            elif num == 5:
                selling = True
                sell_price = close
            
            if selling:
                capital += sell_price * share
                buy_at = 0
                share = 0
                holding = False
                selling = False
                num_trades += 1
            else:
                num += 1

    if holding:
        capital += buy_at * share
        num_trades += 1
    score[ticker] = {'Cap Gain': capital / base_capital - 1}
    